# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096884


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:03<00:47,  1.71s/it]

Rendering models:  10%|█         | 3/30 [00:07<01:02,  2.31s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:43,  1.65s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:29,  1.20s/it]

...

Rendering models:  80%|████████  | 24/30 [00:09<00:00,  6.44it/s]

Rendering models:  83%|████████▎ | 25/30 [00:10<00:00,  6.04it/s]

Rendering models:  87%|████████▋ | 26/30 [00:10<00:00,  4.86it/s]

Rendering models:  90%|█████████ | 27/30 [00:10<00:00,  5.47it/s]

not-logged-in-3b0b929f4475fd276561    0.000263
Jnursssmith5263                       0.001223
persp220                              0.001108
not-logged-in-46a23861c6275e310c67    0.000134
jnarayanbvg                           0.000320
deivad                                0.000264
not-logged-in-c6917d9b19f41e6c2995    0.001305
ICling                                0.000192
merryberryjerrymcflurrythefirst       0.028221
izzyathogwarts                        0.000892
mmcdonough4                           0.001020
not-logged-in-a5721f5f24fded19f7ed    0.002186
WillB2                                0.001189
not-logged-in-5f30b2bb9b7f9d49da41    0.006815
SatanSoo                              0.000437
TheEpicPrimius123                     0.000398
gabrielweiss                          0.002096
Saharisunshine                        0.000197
not-logged-in-6d3e4eb12f2149553110    0.000186
hopefuji                              0.000342
helios1235                            0.000173
Lavadude     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())